In [1]:
import numpy as np, humanfriendly as hf
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,\
     cross_val_score
from sklearn.metrics import mean_squared_error

In [9]:
def get_error(model, Xtest, ytest):
    y_pred = model.predict(Xtest)
    return np.sqrt(mean_squared_error(ytest, y_pred)),model.__class__.__name__

In [10]:
def see_time(note,start):
    end = time.perf_counter()
    elapsed = end - start
    print (note,hf.format_timespan(elapsed, detailed=True))

In [11]:
def get_cross(model, data, target, groups=10):
    return cross_val_score(model, data, target, cv=groups,scoring='neg_mean_squared_error')

In [12]:
X = np.load('data/X_white.npy')
y = np.load('data/y_white.npy')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [13]:
rfr_model = RandomForestRegressor(random_state=0, n_estimators=10)
rfr_model.fit(X_train, y_train)
rmse, rfr_model_name = get_error(rfr_model, X_test, y_test)
print (rfr_model_name + '(rmse):',rmse)

RandomForestRegressor(rmse): 0.6966098665124181


In [14]:
n_est = [100, 500]
boot = [True, False]
params = {'n_estimators': n_est, 'bootstrap': boot}
grid = GridSearchCV(rfr_model, params, cv=5, n_jobs=-1,verbose=1)
start = time.perf_counter()
grid.fit(X_train, y_train)
see_time('training time:',start)
bp = grid.best_params_
print (bp)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   39.2s finished


training time: 50 seconds, 139 milliseconds, 491 microseconds and 700 nanoseconds
{'bootstrap': True, 'n_estimators': 500}


In [15]:
rfr_bpmodel= RandomForestRegressor(**bp, random_state=0)
rfr_bpmodel.fit(X_train, y_train)
rmse, rfr_bpmodel_name = get_error(rfr_bpmodel, X_test, y_test)
print (rfr_bpmodel_name + '(rmse):',rmse)

RandomForestRegressor(rmse): 0.6728175517621279


In [18]:
start = time.perf_counter()
scores = get_cross(rfr_bpmodel, X, y)
see_time('cross-validation rmse:',start)
rmse = np.sqrt(np.mean(scores) * -1)
print (rmse)

cross-validation rmse: 2 minutes, 1 second, 304 milliseconds, 476 microseconds and 200 nanoseconds
0.7183073387927801
